In [1]:
import pandas as pd
import numpy as np
import string
import re

In [2]:
df = pd.read_csv('datasets/patient-doctor_old.csv')
df['Doctor'] = df['Doctor'].apply(lambda text: '<start> ' + text + ' <end>')
df

,Patient,Doctor
0,hello good morning doctor,<start> good morning how are you feeling today...
1,ive been feeling quite anxious lately its been...,<start> i see can you tell me more about what ...
2,i think its mainly related to my job and the p...,<start> stress at work can definitely take a t...
3,i work in a highly demanding environment and i...,<start> that sounds tough do you have any supp...
4,i try to talk to my friends but they dont alwa...,<start> having a strong support system is impo...
...,...,...
1508,i find it difficult to cope and the grief ofte...,<start> coping with grief can be emotionally e...
1509,i havent been very open about my struggles as ...,<start> its common to feel hesitant about shar...
1510,lately i havent been actively practicing selfc...,<start> practicing selfcompassion and engaging...
1511,i havent sought professional help yet im unsur...,<start> seeking professional help such as ther...


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_data(input_texts, target_texts):
    # Create a tokenizer and fit on the input and target texts
    tokenizer = Tokenizer(filters='')
    tokenizer.fit_on_texts(input_texts + target_texts)
    # Convert input and target texts to sequences of integers
    encoder_input_sequences = tokenizer.texts_to_sequences(input_texts)
    decoder_input_sequences = tokenizer.texts_to_sequences(target_texts)
    # Calculate max sequence length
    max_sequence_length = max(max(len(seq) for seq in encoder_input_sequences),
                             max(len(seq) for seq in decoder_input_sequences))
    print('max sequence length:', max_sequence_length)
    # Pad sequences to have the same length
    encoder_input_data = pad_sequences(encoder_input_sequences, maxlen=max_sequence_length, padding='post')
    decoder_input_data = pad_sequences(decoder_input_sequences, maxlen=max_sequence_length, padding='post')
    # Shift target sequences by one time step and convert to one-hot encoding
    decoder_target_data = np.zeros_like(decoder_input_data)
    decoder_target_data[:, :-1] = decoder_input_data[:, 1:]
    decoder_target_data[:, -1] = tokenizer.word_index['<end>']
    # Return preprocessed data and tokenizer
    return encoder_input_data, decoder_input_data, decoder_target_data, tokenizer, max_sequence_length

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation, Input, Embedding, Bidirectional, LSTM, Dense, Attention, Concatenate, Dot, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import vis_utils


def create_model(vocab_size, embedding_dim, hidden_dim, max_sequence_length):
    # Encoder
    encoder_inputs = Input(shape=(max_sequence_length,))
    encoder_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(encoder_inputs)
    encoder_lstm = Bidirectional(LSTM(hidden_dim, return_sequences=True, return_state=True))
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)
    encoder_state_h = Concatenate()([forward_h, backward_h])
    encoder_state_c = Concatenate()([forward_c, backward_c])
    # Decoder
    decoder_inputs = Input(shape=(max_sequence_length,))
    decoder_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(decoder_inputs)
    decoder_lstm = LSTM(hidden_dim*2, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[encoder_state_h, encoder_state_c])
    decoder_outputs = Dropout(0.2)(decoder_outputs)  # Adding Dropout layer for regularization
    # Attention mechanism
    attention = Dot(axes=[2, 2])
    attention_scores = attention([decoder_outputs, encoder_outputs])
    attention_weights = Activation('softmax')(attention_scores)
    context_vector = Dot(axes=[2, 1])([attention_weights, encoder_outputs])
    # Concatenate the context vector and decoder outputs
    decoder_combined_context = Concatenate(axis=-1)([context_vector, decoder_outputs])
    # Dense layer for generating the final output
    decoder_dense = Dense(vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_combined_context)
    # Define the model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    vis_utils.plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)
    model.summary()
    return model

def train_model(model, encoder_input_data, decoder_input_data, decoder_target_data):
    # Hyperparameters
    batch_size = 32
    epochs = 100
    validation_split = 0.2
    patience = 8
    # Define the EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    # train the model
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=validation_split,
              callbacks=[early_stopping])

In [5]:
input_texts = df['Patient'].astype(str).tolist()
target_texts = df['Doctor'].astype(str).tolist()

# Preprocess the data
encoder_input_data, decoder_input_data, decoder_target_data, tokenizer, max_sequence_length = preprocess_data(
    input_texts, target_texts)

max sequence length: 110


In [6]:
vocab_size = len(tokenizer.word_index) + 1
print('Vocab size: ', vocab_size)

# Create the model
model = create_model(vocab_size, embedding_dim = 256, hidden_dim= 256, max_sequence_length = max_sequence_length)

Vocab size:  2741
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 110)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 110, 256)     701696      ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 110)]        0           []                               
                                                                                                  
 bidirectional (Bidirectional)  [(None, 110, 512),   1050624     ['embedding[0][0]']              
                                 (None, 256),                               

In [7]:
# Train the model
train_model(model, encoder_input_data, decoder_input_data, decoder_target_data)

Epoch 1/100
38/38 [==============================] - 178s 4s/step - loss: 6.3030 - accuracy: 0.0436 - val_loss: 5.5855 - val_accuracy: 0.0511
Epoch 2/100
38/38 [==============================] - 168s 4s/step - loss: 5.6515 - accuracy: 0.0688 - val_loss: 5.2863 - val_accuracy: 0.0779
Epoch 3/100
38/38 [==============================] - 169s 4s/step - loss: 5.2046 - accuracy: 0.1097 - val_loss: 4.5350 - val_accuracy: 0.1742
Epoch 4/100
38/38 [==============================] - 167s 4s/step - loss: 4.6537 - accuracy: 0.1985 - val_loss: 3.8493 - val_accuracy: 0.2842
Epoch 5/100
38/38 [==============================] - 169s 4s/step - loss: 4.1295 - accuracy: 0.2671 - val_loss: 3.2728 - val_accuracy: 0.3859
Epoch 6/100
38/38 [==============================] - 168s 4s/step - loss: 3.7029 - accuracy: 0.3227 - val_loss: 2.7993 - val_accuracy: 0.4888
Epoch 7/100
38/38 [==============================] - 157s 4s/step - loss: 3.3567 - accuracy: 0.3643 - val_loss: 2.5574 - val_accuracy: 0.5279
Epoch 

In [10]:
import pickle

# Save the trained model
model.save("models/PD_Bidirectional_model_updated.h5")
model.save("models/PD_Bidirectional_model_updated.keras")

# Save the tokenizer
with open("models/PD_Bidirectional_tokenizer_updated.pkl", "wb") as file:
    pickle.dump(tokenizer, file)